In [32]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from transformers import pipeline
import torch
import os
from tqdm import tqdm
from collections import Counter
import pandas as pd

In [ ]:
def print_formatted(res):
    print(f"sequence: {res['sequence']}")
    print("labels:")
    for label, score in zip(res['labels'], res['scores']):
        print(f"  {label}: {score}")
    print("\n")

def carica_commenti(nome_file):    
    # Costruisce il percorso completo del file
    percorso_file = os.path.join("dati", nome_file)
    
    commenti = {}
    
    try:
        with open(percorso_file, 'r', encoding='utf-8') as file:
            for linea in file:
                linea = linea.strip()  # Rimuove spazi e newline
                
                # Verifica se la linea contiene il formato "nome: commento"
                if ': "' in linea and linea.endswith('"'):
                    # Trova la posizione del primo ': "'
                    pos_separatore = linea.find(': "')
                    
                    nome_utente = linea[:pos_separatore]
                    commento = linea[pos_separatore + 3:-1]  # Rimuove ': "' e l'ultima '"'
                    
                    commenti[nome_utente] = commento
                    
    except FileNotFoundError:
        print(f"File {nome_file} non trovato nella cartella 'dati'")
    except Exception as e:
        print(f"Errore durante la lettura del file: {e}")
    
    return commenti


In [ ]:
# Carica il modello e il tokenizer dal salvataggio locale
local_tokenizer = AutoTokenizer.from_pretrained("multilingual_nli_model")
local_model = AutoModelForSequenceClassification.from_pretrained("multilingual_nli_model")

# Crea una pipeline zero-shot, classifica usando il modello IA in locale
zero_shot_pipe_local = pipeline("zero-shot-classification", model=local_model, tokenizer=local_tokenizer)

# Esempio di inferenza
sequence = "Questo prodotto ha superato le mie aspettative!"
candidate_labels = ["positiva", "negativa", "neutrale"]
result = zero_shot_pipe_local(sequence, candidate_labels=candidate_labels)
print_formatted(result)

Device set to use cpu


sequence: Questo prodotto ha superato le mie aspettative!
labels:
  positiva: 0.9228285551071167
  neutrale: 0.04516180604696274
  negativa: 0.032009612768888474




In [8]:
tot_fb = carica_commenti("fb.txt")

In [ ]:
print(f"Numero di commenti caricati: {len(tot_fb)}")
for nome, commento in tot_fb.items():
    print(f"{nome}: {commento}")
    break 

Numero di commenti caricati: 68
Daniel: C’è da stupirsi? Difficile che diventassero ebrei per compiacere Trump e Netanyahu.


In [10]:
fb_dx = carica_commenti("fb_dx.txt")
print(f"Numero di commenti caricati: {len(fb_dx)}")
fb_sx = carica_commenti("fb_sx.txt")
print(f"Numero di commenti caricati: {len(fb_sx)}")
fb_non_polarizzata = carica_commenti("fb_non_polarizzata.txt")
print(f"Numero di commenti caricati: {len(fb_non_polarizzata)}")
tot_insta = carica_commenti("insta.txt")
print(f"Numero di commenti caricati: {len(tot_insta)}")
insta_dx = carica_commenti("insta_dx.txt")
print(f"Numero di commenti caricati: {len(insta_dx)}")
insta_sx = carica_commenti("insta_sx.txt")
print(f"Numero di commenti caricati: {len(insta_sx)}")
insta_non_polarizzata = carica_commenti("insta_non_polarizzata.txt")
print(f"Numero di commenti caricati: {len(insta_non_polarizzata)}")

Numero di commenti caricati: 20
Numero di commenti caricati: 21
Numero di commenti caricati: 22
Numero di commenti caricati: 87
Numero di commenti caricati: 29
Numero di commenti caricati: 29
Numero di commenti caricati: 40


In [11]:
candidate_labels_topic = ["politica", "economia", "ambiente", "tecnologia", "cultura", "sport"]
candidate_labels_sentiment = ["positivo", "negativo", "neutro"]
candidate_labels_emotion = ["felicità", "tristezza", "rabbia", "paura", "sorpresa", "disgusto"]

In [18]:
def classifica_commenti(candidate_labels, dizionario_commenti):
    """
    Classifica tutti i commenti nel dizionario usando zero-shot classification
    
    Args:
        candidate_labels (list): Lista delle etichette candidate per la classificazione
        dizionario_commenti (dict): Dizionario con {nome_utente: commento}
        
    Returns:
        list: Lista di risultati di classificazione, uno per ogni commento
    """
    risultati = []
    
    for nome_utente, commento in tqdm(dizionario_commenti.items(), desc="Classificando commenti"):
        # Esegue la classificazione zero-shot
        result = zero_shot_pipe_local(commento, candidate_labels=candidate_labels)
        risultati.append(result)
    
    return risultati

In [19]:
###################################################################################################### FACEBOOK
sentiment_fb = classifica_commenti(candidate_labels_sentiment, tot_fb)
topic_fb = classifica_commenti(candidate_labels_topic, tot_fb)
emotion_fb = classifica_commenti(candidate_labels_emotion, tot_fb)

sentiment_fb_dx = classifica_commenti(candidate_labels_sentiment, fb_dx)
topic_fb_dx = classifica_commenti(candidate_labels_topic, fb_dx)
emotion_fb_dx = classifica_commenti(candidate_labels_emotion, fb_dx)

sentiment_fb_sx = classifica_commenti(candidate_labels_sentiment, fb_sx)
topic_fb_sx = classifica_commenti(candidate_labels_topic, fb_sx)
emotion_fb_sx = classifica_commenti(candidate_labels_emotion, fb_sx)

sentiment_fb_non_polarizzata = classifica_commenti(candidate_labels_sentiment, fb_non_polarizzata)
topic_fb_non_polarizzata = classifica_commenti(candidate_labels_topic, fb_non_polarizzata)
emotion_fb_non_polarizzata = classifica_commenti(candidate_labels_emotion, fb_non_polarizzata)

###################################################################################################### INSTAGRAM

sentiment_insta = classifica_commenti(candidate_labels_sentiment, tot_insta)
topic_insta = classifica_commenti(candidate_labels_topic, tot_insta)
emotion_insta = classifica_commenti(candidate_labels_emotion, tot_insta)

sentiment_insta_dx = classifica_commenti(candidate_labels_sentiment, insta_dx)
topic_insta_dx = classifica_commenti(candidate_labels_topic, insta_dx)
emotion_insta_dx = classifica_commenti(candidate_labels_emotion, insta_dx)

sentiment_insta_sx = classifica_commenti(candidate_labels_sentiment, insta_sx)
topic_insta_sx = classifica_commenti(candidate_labels_topic, insta_sx)
emotion_insta_sx = classifica_commenti(candidate_labels_emotion, insta_sx)

sentiment_insta_non_polarizzata = classifica_commenti(candidate_labels_sentiment, insta_non_polarizzata)
topic_insta_non_polarizzata = classifica_commenti(candidate_labels_topic, insta_non_polarizzata)
emotion_insta_non_polarizzata = classifica_commenti(candidate_labels_emotion, insta_non_polarizzata)

Classificando commenti: 100%|██████████| 40/40 [00:22<00:00,  1.78it/s]


In [21]:
def analizza_risultati(lista_risultati):
    """
    Analizza una lista di risultati di classificazione e restituisce le statistiche
    delle etichette più frequenti
    
    Args:
        lista_risultati (list): Lista di risultati di classificazione
        
    Returns:
        list: Lista di tuple (label, percentuale) ordinate per frequenza decrescente
    """
    # Estrae solo le etichette con il punteggio più alto per ogni risultato
    etichette_principali = []
    
    for risultato in lista_risultati:
        # La prima etichetta nella lista 'labels' è quella con il punteggio più alto
        etichetta_principale = risultato['labels'][0]
        etichette_principali.append(etichetta_principale)
    
    # Conta le occorrenze di ogni etichetta
    contatore = Counter(etichette_principali)
    
    # Calcola le percentuali
    totale = len(etichette_principali)
    statistiche = []
    
    for etichetta, count in contatore.items():
        percentuale = (count / totale) * 100
        statistiche.append((etichetta, percentuale))
    
    # Ordina per percentuale decrescente
    statistiche.sort(key=lambda x: x[1], reverse=True)
    
    return statistiche

# Funzione per stampare le statistiche in modo leggibile
def stampa_statistiche(statistiche, titolo="Statistiche"):
    print(f"\n{titolo}:")
    print("-" * 50)
    for etichetta, percentuale in statistiche:
        print(f"{etichetta}: {percentuale:.2f}%")
    print()

In [22]:
fb_sentiment_stats = analizza_risultati(sentiment_fb)
stampa_statistiche(fb_sentiment_stats, "Statistiche Sentiment Facebook")

fb_topic_stats = analizza_risultati(topic_fb)
stampa_statistiche(fb_topic_stats, "Statistiche Topic Facebook")

fb_emotion_stats = analizza_risultati(emotion_fb)
stampa_statistiche(fb_emotion_stats, "Statistiche Emotion Facebook")


Statistiche Sentiment Facebook:
--------------------------------------------------
negativo: 67.65%
positivo: 16.18%
neutro: 16.18%


Statistiche Topic Facebook:
--------------------------------------------------
politica: 38.24%
tecnologia: 19.12%
economia: 17.65%
sport: 8.82%
cultura: 8.82%
ambiente: 7.35%


Statistiche Emotion Facebook:
--------------------------------------------------
disgusto: 47.06%
felicità: 22.06%
tristezza: 14.71%
paura: 8.82%
sorpresa: 5.88%
rabbia: 1.47%



In [23]:
insta_sentiment_stats = analizza_risultati(sentiment_insta)
stampa_statistiche(insta_sentiment_stats, "Statistiche Sentiment Instagram")

insta_topic_stats = analizza_risultati(topic_insta)
stampa_statistiche(insta_topic_stats, "Statistiche Topic Instagram")

insta_emotion_stats = analizza_risultati(emotion_insta)
stampa_statistiche(insta_emotion_stats, "Statistiche Emotion Instagram")


Statistiche Sentiment Instagram:
--------------------------------------------------
negativo: 67.82%
positivo: 21.84%
neutro: 10.34%


Statistiche Topic Instagram:
--------------------------------------------------
politica: 41.38%
cultura: 22.99%
economia: 18.39%
ambiente: 9.20%
tecnologia: 6.90%
sport: 1.15%


Statistiche Emotion Instagram:
--------------------------------------------------
disgusto: 47.13%
felicità: 21.84%
tristezza: 12.64%
sorpresa: 9.20%
paura: 8.05%
rabbia: 1.15%



In [24]:
fb_sx_sentiment_stats = analizza_risultati(sentiment_fb_sx)
stampa_statistiche(fb_sx_sentiment_stats, "Statistiche Sentiment Facebook Sinistra")

fb_sx_topic_stats = analizza_risultati(topic_fb_sx)
stampa_statistiche(fb_sx_topic_stats, "Statistiche Topic Facebook Sinistra")

fb_sx_emotion_stats = analizza_risultati(emotion_fb_sx)
stampa_statistiche(fb_sx_emotion_stats, "Statistiche Emotion Facebook Sinistra")


Statistiche Sentiment Facebook Sinistra:
--------------------------------------------------
negativo: 71.43%
positivo: 14.29%
neutro: 14.29%


Statistiche Topic Facebook Sinistra:
--------------------------------------------------
politica: 47.62%
economia: 23.81%
sport: 9.52%
cultura: 9.52%
tecnologia: 4.76%
ambiente: 4.76%


Statistiche Emotion Facebook Sinistra:
--------------------------------------------------
disgusto: 52.38%
tristezza: 19.05%
felicità: 14.29%
paura: 9.52%
sorpresa: 4.76%



In [25]:
fb_dx_sentiment_stats = analizza_risultati(sentiment_fb_dx)
stampa_statistiche(fb_dx_sentiment_stats, "Statistiche Sentiment Facebook Destra")

fb_dx_topic_stats = analizza_risultati(topic_fb_dx)
stampa_statistiche(fb_dx_topic_stats, "Statistiche Topic Facebook Destra")

fb_dx_emotion_stats = analizza_risultati(emotion_fb_dx)
stampa_statistiche(fb_dx_emotion_stats, "Statistiche Emotion Facebook Destra")


Statistiche Sentiment Facebook Destra:
--------------------------------------------------
negativo: 70.00%
positivo: 20.00%
neutro: 10.00%


Statistiche Topic Facebook Destra:
--------------------------------------------------
politica: 60.00%
cultura: 20.00%
sport: 10.00%
economia: 10.00%


Statistiche Emotion Facebook Destra:
--------------------------------------------------
disgusto: 40.00%
felicità: 25.00%
tristezza: 15.00%
sorpresa: 10.00%
rabbia: 5.00%
paura: 5.00%



In [28]:
fb_non_pol_sentiment_stats = analizza_risultati(sentiment_fb_non_polarizzata)
stampa_statistiche(fb_non_pol_sentiment_stats, "Statistiche Sentiment Facebook Non Polarizzata")

fb_non_pol_topic_stats = analizza_risultati(topic_fb_non_polarizzata)
stampa_statistiche(fb_non_pol_topic_stats, "Statistiche Topic Facebook Non Polarizzata")

fb_non_pol_emotion_stats = analizza_risultati(emotion_fb_non_polarizzata)
stampa_statistiche(fb_non_pol_emotion_stats, "Statistiche Emotion Facebook Non Polarizzata")


Statistiche Sentiment Facebook Non Polarizzata:
--------------------------------------------------
negativo: 59.09%
neutro: 27.27%
positivo: 13.64%


Statistiche Topic Facebook Non Polarizzata:
--------------------------------------------------
tecnologia: 36.36%
politica: 22.73%
economia: 18.18%
ambiente: 13.64%
sport: 9.09%


Statistiche Emotion Facebook Non Polarizzata:
--------------------------------------------------
disgusto: 50.00%
felicità: 27.27%
tristezza: 13.64%
paura: 9.09%



In [29]:
insta_sx_sentiment_stats = analizza_risultati(sentiment_insta_sx)
stampa_statistiche(insta_sx_sentiment_stats, "Statistiche Sentiment Instagram Sinistra")

insta_sx_topic_stats = analizza_risultati(topic_insta_sx)
stampa_statistiche(insta_sx_topic_stats, "Statistiche Topic Instagram Sinistra")

insta_sx_emotion_stats = analizza_risultati(emotion_insta_sx)
stampa_statistiche(insta_sx_emotion_stats, "Statistiche Emotion Instagram Sinistra")


Statistiche Sentiment Instagram Sinistra:
--------------------------------------------------
negativo: 58.62%
positivo: 34.48%
neutro: 6.90%


Statistiche Topic Instagram Sinistra:
--------------------------------------------------
politica: 34.48%
economia: 27.59%
cultura: 13.79%
tecnologia: 13.79%
ambiente: 10.34%


Statistiche Emotion Instagram Sinistra:
--------------------------------------------------
disgusto: 27.59%
felicità: 27.59%
sorpresa: 20.69%
tristezza: 13.79%
paura: 6.90%
rabbia: 3.45%



In [30]:
insta_dx_sentiment_stats = analizza_risultati(sentiment_insta_dx)
stampa_statistiche(insta_dx_sentiment_stats, "Statistiche Sentiment Instagram Destra")

insta_dx_topic_stats = analizza_risultati(topic_insta_dx)
stampa_statistiche(insta_dx_topic_stats, "Statistiche Topic Instagram Destra")

insta_dx_emotion_stats = analizza_risultati(emotion_insta_dx)
stampa_statistiche(insta_dx_emotion_stats, "Statistiche Emotion Instagram Destra")


Statistiche Sentiment Instagram Destra:
--------------------------------------------------
negativo: 72.41%
neutro: 17.24%
positivo: 10.34%


Statistiche Topic Instagram Destra:
--------------------------------------------------
politica: 65.52%
cultura: 17.24%
economia: 6.90%
tecnologia: 3.45%
ambiente: 3.45%
sport: 3.45%


Statistiche Emotion Instagram Destra:
--------------------------------------------------
disgusto: 51.72%
sorpresa: 17.24%
felicità: 13.79%
paura: 13.79%
tristezza: 3.45%



In [31]:
insta_non_pol_sentiment_stats = analizza_risultati(sentiment_insta_non_polarizzata)
stampa_statistiche(insta_non_pol_sentiment_stats, "Statistiche Sentiment Instagram Non Polarizzata")

insta_non_pol_topic_stats = analizza_risultati(topic_insta_non_polarizzata)
stampa_statistiche(insta_non_pol_topic_stats, "Statistiche Topic Instagram Non Polarizzata")

insta_non_pol_emotion_stats = analizza_risultati(emotion_insta_non_polarizzata)
stampa_statistiche(insta_non_pol_emotion_stats, "Statistiche Emotion Instagram Non Polarizzata")


Statistiche Sentiment Instagram Non Polarizzata:
--------------------------------------------------
negativo: 72.50%
positivo: 20.00%
neutro: 7.50%


Statistiche Topic Instagram Non Polarizzata:
--------------------------------------------------
cultura: 30.00%
politica: 30.00%
economia: 20.00%
tecnologia: 10.00%
ambiente: 10.00%


Statistiche Emotion Instagram Non Polarizzata:
--------------------------------------------------
disgusto: 57.50%
felicità: 20.00%
tristezza: 17.50%
sorpresa: 2.50%
paura: 2.50%



In [ ]:
# Funzione per convertire le statistiche in un dizionario
def stats_to_dict(stats, labels):
    """
    Converte le statistiche in un dizionario con tutte le label
    """
    result = {label: 0.0 for label in labels}  # Inizializza tutte le label a 0
    for label, percentage in stats:
        result[label] = percentage
    return result

# Crea il DataFrame con tutte le statistiche
sentiment_data = []
topic_data = []
emotion_data = []

# Facebook
sentiment_data.append(("FB Total", stats_to_dict(fb_sentiment_stats, candidate_labels_sentiment)))
topic_data.append(("FB Total", stats_to_dict(fb_topic_stats, candidate_labels_topic)))
emotion_data.append(("FB Total", stats_to_dict(fb_emotion_stats, candidate_labels_emotion)))

# Instagram
sentiment_data.append(("IG Total", stats_to_dict(insta_sentiment_stats, candidate_labels_sentiment)))
topic_data.append(("IG Total", stats_to_dict(insta_topic_stats, candidate_labels_topic)))
emotion_data.append(("IG Total", stats_to_dict(insta_emotion_stats, candidate_labels_emotion)))

# Facebook Sinistra
sentiment_data.append(("FB Sinistra", stats_to_dict(fb_sx_sentiment_stats, candidate_labels_sentiment)))
topic_data.append(("FB Sinistra", stats_to_dict(fb_sx_topic_stats, candidate_labels_topic)))
emotion_data.append(("FB Sinistra", stats_to_dict(fb_sx_emotion_stats, candidate_labels_emotion)))

# Facebook Destra
sentiment_data.append(("FB Destra", stats_to_dict(fb_dx_sentiment_stats, candidate_labels_sentiment)))
topic_data.append(("FB Destra", stats_to_dict(fb_dx_topic_stats, candidate_labels_topic)))
emotion_data.append(("FB Destra", stats_to_dict(fb_dx_emotion_stats, candidate_labels_emotion)))

# Facebook Non Polarizzata
sentiment_data.append(("FB Non Polarizzata", stats_to_dict(fb_non_pol_sentiment_stats, candidate_labels_sentiment)))
topic_data.append(("FB Non Polarizzata", stats_to_dict(fb_non_pol_topic_stats, candidate_labels_topic)))
emotion_data.append(("FB Non Polarizzata", stats_to_dict(fb_non_pol_emotion_stats, candidate_labels_emotion)))

# Instagram Sinistra
sentiment_data.append(("IG Sinistra", stats_to_dict(insta_sx_sentiment_stats, candidate_labels_sentiment)))
topic_data.append(("IG Sinistra", stats_to_dict(insta_sx_topic_stats, candidate_labels_topic)))
emotion_data.append(("IG Sinistra", stats_to_dict(insta_sx_emotion_stats, candidate_labels_emotion)))

# Instagram Destra
sentiment_data.append(("IG Destra", stats_to_dict(insta_dx_sentiment_stats, candidate_labels_sentiment)))
topic_data.append(("IG Destra", stats_to_dict(insta_dx_topic_stats, candidate_labels_topic)))
emotion_data.append(("IG Destra", stats_to_dict(insta_dx_emotion_stats, candidate_labels_emotion)))

# Instagram Non Polarizzata
sentiment_data.append(("IG Non Polarizzata", stats_to_dict(insta_non_pol_sentiment_stats, candidate_labels_sentiment)))
topic_data.append(("IG Non Polarizzata", stats_to_dict(insta_non_pol_topic_stats, candidate_labels_topic)))
emotion_data.append(("IG Non Polarizzata", stats_to_dict(insta_non_pol_emotion_stats, candidate_labels_emotion)))

# Creare i DataFrame
df_sentiment = pd.DataFrame([data[1] for data in sentiment_data], 
                           index=[data[0] for data in sentiment_data])

df_topic = pd.DataFrame([data[1] for data in topic_data], 
                       index=[data[0] for data in topic_data])

df_emotion = pd.DataFrame([data[1] for data in emotion_data], 
                         index=[data[0] for data in emotion_data])

# Visualizzare i DataFrame
print("SENTIMENT ANALYSIS:")
print(df_sentiment.round(2))
print("\n" + "="*50 + "\n")

print("TOPIC ANALYSIS:")
print(df_topic.round(2))
print("\n" + "="*50 + "\n")

print("EMOTION ANALYSIS:")
print(df_emotion.round(2))

SENTIMENT ANALYSIS:
                    positivo  negativo  neutro
FB Total               16.18     67.65   16.18
IG Total               21.84     67.82   10.34
FB Sinistra            14.29     71.43   14.29
FB Destra              20.00     70.00   10.00
FB Non Polarizzata     13.64     59.09   27.27
IG Sinistra            34.48     58.62    6.90
IG Destra              10.34     72.41   17.24
IG Non Polarizzata     20.00     72.50    7.50


TOPIC ANALYSIS:
                    politica  economia  ambiente  tecnologia  cultura  sport
FB Total               38.24     17.65      7.35       19.12     8.82   8.82
IG Total               41.38     18.39      9.20        6.90    22.99   1.15
FB Sinistra            47.62     23.81      4.76        4.76     9.52   9.52
FB Destra              60.00     10.00      0.00        0.00    20.00  10.00
FB Non Polarizzata     22.73     18.18     13.64       36.36     0.00   9.09
IG Sinistra            34.48     27.59     10.34       13.79    13.79   0.00
